# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

In [181]:
import requests
from bs4 import BeautifulSoup as soup
import re
import time
import pandas as pd
import json

In [182]:
def get_posts(page_link, keyword, post_list):
    for post in soup(requests.get(page_link, params={'q': keyword,}).text, 'html.parser').find_all('a', class_='post__title_link'):
        post_link = post.get('href')
        post = soup(requests.get(post_link).text, 'html.parser')
        post_list.append({'keyword': keyword,
                          'date': post.find('span', class_='post__time').get('data-time_published'), 
                          'header': post.find('span', class_='post__title-text').text, 
                          'post_link': post_link, 
                          'post_text': post.find(id="post-content-body").text})
    return post_list

def get_search_result(keywords):
    post_list = []
    for keyword in keywords:
        last_page = (int(re.search(r'(page)(\d+)', 
                                   soup(requests.get('https://habr.com/ru/search/', params={'q': keyword,})
                                   .text, 'html.parser')
                                   .find('a', class_='toggle-menu__item-link_bordered')
                                   .get('href')).group(2)))
        for page in range(1, last_page + 1):
            get_posts(f'https://habr.com/ru/search/', keyword, post_list) if page == 1 else get_posts(f'https://habr.com/ru/search/page{page}/', keyword, post_list)
    return post_list

keywords = ['python', 'парсинг']
df = pd.DataFrame(get_search_result(keywords))
df

,keyword,date,header,post_link,post_text
0,python,2020-04-21T15:35Z,"Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/t/498364/,"\r\n20 апреля 2020 года, спустя почти десять л..."
1,python,2020-03-03T10:22Z,В начале этого года Python сместил Java и стал...,https://habr.com/ru/company/itsumma/news/t/490...,"Согласно отчету RedMonk за январь 2020 года, P..."
2,python,2020-06-03T06:00Z,Вышла версия 0.0.2 snakeware — дистрибутива Li...,https://habr.com/ru/news/t/505096/,\n\r\n31 мая 2020 года разработчик Джош Мур (J...
3,python,2019-10-31T07:02Z,Создатель Python Гвидо ван Россум ушел из Drop...,https://habr.com/ru/news/t/473926/,\n\r\nСоздатель языка программирования Python ...
4,python,2012-07-09T06:21Z,Мысли о Python 3,https://habr.com/ru/post/147281/,Предлагаю вашему вниманю пересказ замечательно...
...,...,...,...,...,...
1994,парсинг,2014-08-21T15:41Z,mysqlnd — проводник между PHP и MySQL,https://habr.com/ru/company/mailru/blog/234125/,\n\r\nРасширение mysqlnd появилось ещё в PHP 5...
1995,парсинг,2014-08-25T07:41Z,Несколько интересностей и полезностей от веб-р...,https://habr.com/ru/post/234279/,"* Надеюсь, ilusha_sergeevich не обвинит меня ..."
1996,парсинг,2014-09-02T09:26Z,Приглашаем на Moscow Django MeetUp № 22,https://habr.com/ru/company/mailru/blog/235227/,"\n\r\nДамы и господа, в четверг в офисе Mail.R..."
1997,парсинг,2014-09-04T07:10Z,Основные отличия Java IO и Java NIO,https://habr.com/ru/post/235585/,Когда я начал изучать стандартный ввод/вывод в...


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [190]:
def get_email_leak_status(emails):
    list_of_leaks = []
    for email in emails:
        for leak in requests.post('https://digibody.avast.com/v1/web/leaks', data=json.dumps({'email': email})).json()['value']:
            list_of_leaks.append({
                'email': email,
                'date_leak': leak['leak_info']['date'],
                'leak_source': leak['domain'],
                'leak_description': leak['leak_info']['description'],
            })
    return pd.DataFrame(list_of_leaks)

emails = ['aaogoltcov@mail.ru', 'alexey.ogoltsov@outlook.com',]
get_email_leak_status(emails)

,email,date_leak,leak_source,leak_description
0,aaogoltcov@mail.ru,1558569600000,livejournal.com,"In 2017, social network LiveJournal's database..."
1,aaogoltcov@mail.ru,1549411200000,,"On January 7, 2019, an online user named Sanix..."
2,aaogoltcov@mail.ru,1549411200000,,"On January 7, 2019, an online user named Sanix..."
3,aaogoltcov@mail.ru,1571270400000,zynga.com,"In September 2019, the game developer Zynga wa..."
4,aaogoltcov@mail.ru,1509753600000,myheritage.com,"In October 2017, a customer database belonging..."
5,aaogoltcov@mail.ru,1548720000000,,"On January 7, 2019, an online user named Sanix..."
6,aaogoltcov@mail.ru,1576195200000,,This source has been marked as sensitive due t...
7,aaogoltcov@mail.ru,1477958400000,qip.ru,"In 2011, Russian instant messaging service pro..."
8,aaogoltcov@mail.ru,1575504000000,,This source has been marked as sensitive due t...
9,alexey.ogoltsov@outlook.com,1509753600000,myheritage.com,"In October 2017, a customer database belonging..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)```
GROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
```

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

In [189]:
response = requests.get('https://api.vk.com/method/wall.get?', params = {
        'access_token': 'FILL ACCESS TOKEN',
        'v': 5.122,
        'domain': 'netology',
        'count': 50,
}).json()['response']['items']
list_of_posts = []
for post in response:
    list_of_posts.append({
        'post_date': post['date'],
        'post_text': post['text'],
    })
pd.DataFrame(list_of_posts)

,post_date,post_text
0,1598022840,А ю рэди? 🕺 На следующей неделе учимся выступа...
1,1598455920,Хорошие новости: мы почти нашли ведущего для н...
2,1598428200,"Ой, сделайте меня стройнее, поставьте рядом Дж..."
3,1598370120,"Казалось бы, общение — это просто. Кто-то гово..."
4,1598345789,"Любимая работа, амбициозные проекты, полное по..."
5,1598282460,"Буквы разные писать, тонким пёрышком в тетрадь..."
6,1598254680,SMM — это не только про креатив. Здесь важен р...
7,1598188440,Подкасты — отличный способ узнавать что-то нов...
8,1598170320,Вместе с командой приложения для проектировани...
9,1598102100,*партнерский пост* \n \n17-18 сентября пройдёт...


#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
